In [1]:
import os
import random
from datetime import datetime
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType, TimestampType, IntegerType
import seaborn as sns
import d5_solutions as sol

In [19]:
def preprocess_cdrs_using_spark(file_or_folder=None, number_of_users_to_sample=None,
                                output_csv=None, date_format='%Y%m%d%H%M%S',
                                debug_mode=True, loc_file=None, save_to_csv=False):
    """
    In this function, we perfom some basic preprocessing such as below:
    1. rename columns
    2. change some data types
    3. Add location details
    Eventually, we will sample the data to use for our analysis
    :param data_folder:
    :param output_csv_for_sample_users:
    :return:
    """

    # create SparkSession object
    spark = SparkSession.builder.master("local[8]").appName("data_processor").getOrCreate()

    # read data with spark
    df = spark.read.csv(path=file_or_folder, header=True)

    # repartition to speed up
    df = df.repartition(10)

    # if just testing/debugging, pick only a small dataset
    if debug_mode:
        dfs = df.sample(fraction=0.01)
        df = dfs

    # rename columns to remove space and replace with underscore
    df2 = (df.withColumnRenamed("cdr datetime", "cdrDatetime")
        .withColumnRenamed("calling phonenumber2", "phoneNumber")
        .withColumnRenamed("last calling cellid", "cellId")
        .withColumnRenamed("call duration", "cellDuration"))

    # drop cdr type column
    df3 = df2.drop('cdr type')

    # Use Spark UDF to add date and datetime
    add_datetime = udf(lambda x: datetime.strptime(x, date_format), TimestampType())
    add_date = udf(lambda x: datetime.strptime(x, date_format), DateType())

    # create timestamp
    df4 = df3.withColumn('datetime', add_datetime(col('cdrDatetime')))
    df5 = df4.withColumn('date', add_date(col('cdrDatetime')))

    # lets make sure we dont have any null phoneNumbers
    df6 = df5.filter(df5['phoneNumber'].isNotNull())

    # Lets merge with location details using cellId from CDRs and also
    # cellID on the other
    dfLoc = pd.read_csv(loc_file)
    dfLoc.rename(columns={'cell_id': 'cellId'}, inplace=True)
    sdfLoc = spark.createDataFrame(dfLoc)
    df7 = df6.join(sdfLoc, on='cellId', how='inner')
    
    return df7

#     # select nsample users to work with
#     all_users = df7.select('phoneNumber').distinct().collect()

#     # randomly select users using filter statement
#     random_user_numbers = [i['phoneNumber'] for i in random.choices(all_users, k=number_of_users_to_sample)]

#     # select only our random user data
#     dfu = df7.filter(df7['phoneNumber'].isin(random_user_numbers))

#     # save to CSV if necessary
#     if save_to_csv:
#         dfu.coalesce(1).write.csv(path=output_csv, header=True)
#     else:
#         return dfu


In [2]:
def combine_selected_csv_files(folder_with_csv_files=None, number_to_save=None, out_csv_file=None):
    """
    Save a sample of the small CSV files into a CSV file for exploration.
    Please test this with very few files to avoid wasting time
    :param folder_with_csv_files:
    :param number_to_save:
    :return:
    """

    # get a list of CSV file using os module listdir() function
    files = os.listdir(folder_with_csv_files)
    
    # create a list to hold pandas dataframes
    df_lst = []
    
    #create a counter variable whcih will help you stop
    # the loop when you reach the required number of files
    cnt = 0
    for f in files:
        if f.endswith('csv'):
            fpath = os.path.join(folder_with_csv_files, f)
            df = pd.read_csv(fpath)
            # append this df to the list of dfs above
            df_lst.append(df)
    
            # increment the counter variable
            cnt += 1
    
            # stop the loop using break statement when you have
            # processes the required number of files
            # as defined by number_to_save
            if cnt == number_to_save:
                break
    
    # use pandas function concat() like this: pd.concat()
    # to concatenate all the dfs in the list
    df = pd.concat(df_lst)
    
    # save your new dataframe
    df.to_csv(out_csv_file, index=False)


In [3]:
def explore_data(df=None, output_plot_file=None, output_heatmap=None):
    """
    For quick examination of user activity, lets generate
    user call count and do a simple plot.
    """
    # Number of days in data
    dates_rows = df.select('date').distinct().collect()
    sorted_dates = sorted([i['date'] for i in dates_rows])
    diff = sorted_dates[-1] - sorted_dates[0]
    num_days = diff.days

    # call count by hour
    add_hr = udf(lambda x: x.hour, IntegerType())
    add_wkday = udf(lambda x: x.weekday(), IntegerType())
    day_dict = {0: 'Mon', '1': 'Tue', '2': 'Wed', 3: 'Thurs', 4: 'Frid', 5: 'Sat', 6: 'Sun'}

    dfHr = df.withColumn('hr', add_hr(col('datetime')))
    dfHr2 = dfHr.withColumn('wkday', add_wkday(col('datetime')))
    dfWkDay = dfHr2.groupBy('wkday', 'hr').count().toPandas()
    dfWkDay['weekDay'] = dfWkDay.apply(add_weekdays, args=(day_dict,), axis=1)
    dfWkDay.drop(labels=['wkday'], axis=1, inplace=True)
    dfWkDayPivot = dfWkDay.pivot(index='weekDay', columns='hr', values='count')
    d = dfWkDayPivot.reset_index()
    ax = sns.heatmap(d)
    ax.get_figure().savefig(output_heatmap)

    # group user and count number of events
    # convert resulting spark dataframe to pandas
    dfGroup = df.groupBy('phoneNumber').count().toPandas()

    # create a distribution plot of user call count using
    # seaborn
    ax = sns.distplot(dfGroup['count'])

    # save plot as png file
    ax.get_figure().savefig(output_plot_file)

    # report average number calls per day for each user
    dfGroupDay = df.groupBy('phoneNumber', 'date').count().toPandas()

    # get mean and median
    mean = dfGroupDay['count'].mean()
    median = dfGroupDay['count'].median()

    # data duration
    return mean, median, num_days

In [11]:
dataFolder = '../../day5-case-study/cdrs/'
users_call_cnt_plt = '../../day5-case-study/userCallCnt.png'
heatMap = '../../day5-case-study/dayHrHeatmap.png'

In [20]:
dfs = preprocess_cdrs_using_spark(file_or_folder=dataFolder, number_of_users_to_sample=50000, 
                                  date_format='%Y%m%d%H%M%S', 
                                  loc_file='../../day5-case-study/cellTowers/staggered-cell-locs.csv',
                                 save_to_csv=False, debug_mode=True)

In [ ]:
mean, median, num_days, hrDayCnts = sol.explore_data(df=dfs, output_plot_file=users_call_cnt_plt , 
             output_heatmap=heatMap)

In [21]:
dfs.head()

Row(cellId='20532.0', cdrDatetime='20180711204136', cellDuration=None, phoneNumber='8204330690229196471', datetime=datetime.datetime(2018, 7, 11, 20, 41, 36), date=datetime.date(2018, 7, 11), site_id='s91', lon=28.885225, lat=-19.060772)

In [23]:
grpByUser = dfs.rdd.map(lambda x: (x['phoneNumber'], x))

('8204330690229196471',
 Row(cellId='20532.0', cdrDatetime='20180711204136', cellDuration=None, phoneNumber='8204330690229196471', datetime=datetime.datetime(2018, 7, 11, 20, 41, 36), date=datetime.date(2018, 7, 11), site_id='s91', lon=28.885225, lat=-19.060772))

In [26]:
grpByUser2 = grpByUser.groupByKey()
grpByUser2.first()

('217873087185206285', <pyspark.resultiterable.ResultIterable at 0x1a30c7e828>)

In [ ]:
grpByUser = dfs.rdd.map(lambda x: (x['phoneNumber'], x))
grpByUser2 = grpByUser.groupByKey()

# select nsample users to work with
all_users = df7.select('phoneNumber').distinct().collect()

# randomly select users using filter statement
random_user_numbers = [i['phoneNumber'] for i in random.choices(all_users, k=number_of_users_to_sample)]

# select only our random user data

dfu = df7.filter(df7['phoneNumber'].isin(random_user_numbers))

In [27]:
# select nsample users to work with
all_users = dfs.select('phoneNumber').distinct().collect()

# randomly select users using filter statement
random_user_numbers = [i['phoneNumber'] for i in random.choices(all_users, k=100)]

In [29]:
dfu = grpByUser2.filter(lambda x: x[0] in random_user_numbers)

In [30]:
dfu.first()

('5030518197241310850',
 <pyspark.resultiterable.ResultIterable at 0x1a421a5198>)

In [32]:
k = dfu.keys().collect()

In [33]:
len(k)

22